# CarND-Behavioral-Cloning-P3

## Read from csv file

In [2]:
import csv
import cv2
import numpy as np

def read_data(path):
    print("Start read training data from path: " + path)
    driving_log_path = path + "driving_log.csv"
    lines = []
    
    with open(driving_log_path) as csv_file:
        reader = csv.reader(csv_file)
        for line in reader:
            lines.append(line)
    print("Read from csv file done!")
    
    images = []
    measurements = []
    for i, line in enumerate(lines):
        if i == 0:
            continue
        # read in center images
        image_path = line[0]
        file_name = image_path.split("/")[-1]
        file_path = path + "IMG/"
        data_path = file_path + file_name
        image = cv2.imread(data_path)
        images.append(image)
        # read in angles
        measurement = float(line[-1])
        measurements.append(measurement)
    print("Extract center images and angle measurements from file done! Get {0} samples".format(len(images)))
    
    # Data augment
    augment_images, augment_measurements = [], []
    for image, measurement in zip(images, measurements):
        augment_images.append(image)
        augment_measurements.append(measurement)
        augment_images.append(cv2.flip(image, 1))
        augment_measurements.append(measurement * -1.0)
    print("Data augment done!")
    print()

    return augment_images, augment_measurements
    

def merge_data(path, X_train = None, y_train = None):
    custom_images, custom_measurements = read_data(path)
    if X_train is None:
        print("Create data sets...")
        X_train = np.array(custom_images)
        y_train = np.array(custom_measurements)
    else:   
        print("Concat data sets...")
        X_train = np.concatenate((X_train, np.array(custom_images)))
        y_train = np.concatenate((y_train, np.array(custom_measurements)))
    return X_train, y_train

# Read from data/
X_train, y_train = merge_data("data/")
# X_train, y_train = merge_data("custom_data/", X_train, y_train)
X_train, y_train = merge_data("custom_data_0211_1/", X_train, y_train)
X_train, y_train = merge_data("custom_data_0211_2/", X_train, y_train)


print("Done! Get {0} samples.".format(len(X_train)))

Start read training data from path: data/
Read from csv file done!


Extract center images and angle measurements from file done! Get 8036 samples


Data augment done!

Create data sets...


Start read training data from path: custom_data_0211_1/
Read from csv file done!


Extract center images and angle measurements from file done! Get 1042 samples
Data augment done!

Concat data sets...


Start read training data from path: custom_data_0211_2/
Read from csv file done!


Extract center images and angle measurements from file done! Get 5070 samples


Data augment done!

Concat data sets...


Done! Get 28296 samples.


### Test the training data

In [2]:
print("X_train shape: " + str(X_train.shape))
print("y_train shape: " + str(y_train.shape))

X_train shape: (28296, 160, 320, 3)
y_train shape: (28296,)


## Train the model

In [12]:
# # Train model
# from keras.models import Sequential, Model
# from keras.layers import Lambda, Flatten, Dense
# 
# print("Start training the model...")
# 
# model = Sequential()
# model.add(Flatten(input_shape=(160,320,3)))
# model.add(Dense(1))
# 
# model.compile(loss='mse', optimizer='adam')
# model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=8)
# 
# model.save("model.h5")
# print("Training done, save model to model.h5")

Turns out the flow is working. But it is not working fine. To improve the model we can add preprocessing steps to the training process. 

Add the Navida power architecture to train the model.

In [4]:
# Train model
import os  

from keras.models import Sequential, Model
from keras.layers import Lambda, Flatten, Dense, Cropping2D, Convolution2D

print("Start training the model...")

model = Sequential()
model.add(Lambda(lambda x : (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

model_name = "models/model_0211_1.h5"
model.save(model_name)
print("Training done, save model to {0}".format(model_name))




# Play beep alert after training done!
import pygame
import time

pygame.mixer.init()
track=pygame.mixer.music.load("/Users/dianping/sdc_workspace/alert.mp3") #可以播放.mp3 .wav等多种格式的音频文件
loop_time = 5

for i in range(loop_time):
    pygame.mixer.music.play()
    time.sleep(10)  


Using TensorFlow backend.


Start training the model...


Train on 22636 samples, validate on 5660 samples
Epoch 1/5


   32/22636 [..............................] - ETA: 743s - loss: 884.7407

   64/22636 [..............................] - ETA: 512s - loss: 860.0013

...

22592/22636 [============================>.] - ETA: 0s - loss: 17.3461

22624/22636 [============================>.] - ETA: 0s - loss: 17.3372

22636/22636 [==============================] - 240s - loss: 17.3310 - val_loss: 44.9135


Training done, save model to model_0211_1.h5


32512

To learn from previous model. We use transfer learning here to save time and compute resource.

In [8]:
# Play beep alert after training done!
import pygame
import time

pygame.mixer.init()
track=pygame.mixer.music.load("/Users/dianping/sdc_workspace/alert.mp3") #可以播放.mp3 .wav等多种格式的音频文件
loop_time = 5

for i in range(loop_time):
    pygame.mixer.music.play()
    time.sleep(10)  


ImportError: No module named 'pygame'